# PKGS

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

# Creación por chunks

In [2]:
data = pd.read_excel('PruebaDatosVaRAjustado.xlsx', sheet_name=1).set_index('Date').sort_index()
data.tail()

,Bid,Ask,Bid.1,Ask.1
Date,,,,
2025-02-25,110.515625,110.531250,117.62500,117.65625
2025-02-26,110.765625,110.796875,118.18750,118.21875
2025-02-27,110.812500,110.828125,117.78125,117.81250
2025-02-28,111.171875,111.187500,118.34375,118.40625
2025-03-03,111.640625,111.656250,119.18750,119.21875


In [3]:
assets = ['TB10Y', 'TB30Y']

data.columns = [f"{col}_{asset}" for asset, (col1, col2) in zip(assets, zip(data.columns[::2], data.columns[1::2])) for col in (col1, col2)]
data.columns = data.columns.str.replace(r"(Bid|Ask).*?_","\\1_", regex=True)
data.tail()

,Bid_TB10Y,Ask_TB10Y,Bid_TB30Y,Ask_TB30Y
Date,,,,
2025-02-25,110.515625,110.531250,117.62500,117.65625
2025-02-26,110.765625,110.796875,118.18750,118.21875
2025-02-27,110.812500,110.828125,117.78125,117.81250
2025-02-28,111.171875,111.187500,118.34375,118.40625
2025-03-03,111.640625,111.656250,119.18750,119.21875


In [4]:
for i in range(len(assets)):
    data[f"Mid_{assets[i]}"] = (data[f"Ask_{assets[i]}"] + data[f"Bid_{assets[i]}"]) / 2
    data[f"Spread_{assets[i]}"] = (data[f"Ask_{assets[i]}"] - data[f"Bid_{assets[i]}"]) / data[f"Mid_{assets[i]}"]
    data[f"Rend_{assets[i]}"] = data[f"Mid_{assets[i]}"].pct_change().dropna()

data.tail()

,Bid_TB10Y,Ask_TB10Y,Bid_TB30Y,Ask_TB30Y,Mid_TB10Y,Spread_TB10Y,Rend_TB10Y,Mid_TB30Y,Spread_TB30Y,Rend_TB30Y
Date,,,,,,,,,,
2025-02-25,110.515625,110.531250,117.62500,117.65625,110.523438,0.000141,0.006403,117.640625,0.000266,0.012098
2025-02-26,110.765625,110.796875,118.18750,118.21875,110.781250,0.000282,0.002333,118.203125,0.000264,0.004782
2025-02-27,110.812500,110.828125,117.78125,117.81250,110.820312,0.000141,0.000353,117.796875,0.000265,-0.003437
2025-02-28,111.171875,111.187500,118.34375,118.40625,111.179688,0.000141,0.003243,118.375000,0.000528,0.004908
2025-03-03,111.640625,111.656250,119.18750,119.21875,111.648438,0.000140,0.004216,119.203125,0.000262,0.006996


In [5]:
titulos = [100000000, 90000000]

posiciones = [data[f"Mid_{assets[i]}"].iloc[-1] * titulos[i] for i in range(len(assets))]

weights = posiciones / sum(posiciones)

rend_port = data[[f"Rend_{asset}" for asset in assets]].dot(weights).dropna()

long = False
conf = 99

var = np.percentile(rend_port.values,100-conf) if long else np.percentile(rend_port.values,conf)
cvar = rend_port[rend_port < var].mean() if long else rend_port[rend_port > var].mean()

pd.set_option('display.float_format', '{:,.4f}'.format)

var_df = pd.DataFrame({"Métrica": ["VaR", "CVaR"], "Porcentual (%)": [var * 100, cvar * 100], "Monetario ($)": [var * sum(posiciones), cvar * sum(posiciones)]})
var_df

,Métrica,Porcentual (%),Monetario ($)
0,VaR,1.6074,"351,914,316.5583"
1,CVaR,1.8068,"395,554,861.5311"


In [6]:
CL = [np.percentile([data[f"Spread_{asset}"]], 99) for asset in assets]
CL_m = [CL[i] * posiciones[i] for i in range(len(assets))]
CL_df = pd.DataFrame({"Activo": assets, "Porcentual (%)": np.array(CL)*100, "Monetario ($)": CL_m})
CL_df

,Activo,Porcentual (%),Monetario ($)
0,TB10Y,0.0290,"3,240,128.4630"
1,TB30Y,0.0793,"8,511,584.1758"


In [7]:
var_apl = pd.DataFrame({"Métrica": ["VaR ApL", "CVaR ApL"], "Porcentual (%)": [(var + sum(CL))*100, (cvar + sum(CL))*100], "Monetario ($)": [var * sum(posiciones) + sum(CL_m), cvar * sum(posiciones) + sum(CL_m)]})
var_apl

,Métrica,Porcentual (%),Monetario ($)
0,VaR ApL,1.7158,"363,666,029.1972"
1,CVaR ApL,1.9151,"407,306,574.1699"


# Función

In [8]:
def VaR_ApL(data, assets, titulos, long, conf):
     data.columns = [f"{col}_{asset}" for asset, (col1, col2) in zip(assets, zip(data.columns[::2], data.columns[1::2])) for col in (col1, col2)]
     data.columns = data.columns.str.replace(r"(Bid|Ask).*?_","\\1_", regex=True)

     for i in range(len(assets)):
          data[f"Mid_{assets[i]}"] = (data[f"Ask_{assets[i]}"] + data[f"Bid_{assets[i]}"]) / 2
          data[f"Spread_{assets[i]}"] = (data[f"Ask_{assets[i]}"] - data[f"Bid_{assets[i]}"]) / data[f"Mid_{assets[i]}"]
          data[f"Rend_{assets[i]}"] = data[f"Mid_{assets[i]}"].pct_change().dropna()
     
     posiciones = [data[f"Mid_{assets[i]}"].iloc[-1] * titulos[i] for i in range(len(assets))]
     weights = posiciones / sum(posiciones)
     rend_port = data[[f"Rend_{asset}" for asset in assets]].dot(weights).dropna()

     var = np.percentile(rend_port.values,100-conf) if long else np.percentile(rend_port.values,conf)
     cvar = rend_port[rend_port < var].mean() if long else rend_port[rend_port > var].mean()
     var_df = pd.DataFrame({"Métrica": ["VaR", "CVaR"], "Porcentual (%)": [var * 100, cvar * 100], "Monetario ($)": [var * sum(posiciones), cvar * sum(posiciones)]})

     CL = [np.percentile([data[f"Spread_{asset}"]], 99) for asset in assets]
     CL_m = [CL[i] * posiciones[i] for i in range(len(assets))]
     CL_df = pd.DataFrame({"Activo": assets, "Porcentual (%)": np.array(CL)*100, "Monetario ($)": CL_m})

     var_apl = pd.DataFrame({"Métrica": ["VaR ApL", "CVaR ApL"], "Porcentual (%)": [(var + sum(CL))*100, (cvar + sum(CL))*100], "Monetario ($)": [var * sum(posiciones) + sum(CL_m), cvar * sum(posiciones) + sum(CL_m)]})
     return display(var_df), display(CL_df), display(var_apl)


In [9]:
data = pd.read_excel('PruebaDatosVaRAjustado.xlsx', sheet_name=1).set_index('Date').sort_index()
assets = ['TB10Y', 'TB30Y']
titulos = [100000000, 90000000]
long = False
conf = 99

VaR_ApL(data, assets, titulos, long, conf)

,Métrica,Porcentual (%),Monetario ($)
0,VaR,1.6074,"351,914,316.5583"
1,CVaR,1.8068,"395,554,861.5311"


,Activo,Porcentual (%),Monetario ($)
0,TB10Y,0.0290,"3,240,128.4630"
1,TB30Y,0.0793,"8,511,584.1758"


,Métrica,Porcentual (%),Monetario ($)
0,VaR ApL,1.7158,"363,666,029.1972"
1,CVaR ApL,1.9151,"407,306,574.1699"


(None, None, None)

# Optimización chat

In [10]:
import numpy as np
import pandas as pd

def VaR_ApL(data, assets, titulos, long, conf):
    # Renombrar columnas eliminando caracteres innecesarios
    data.columns = [f"{col}_{asset}" for asset, (col1, col2) in zip(assets, zip(data.columns[::2], data.columns[1::2])) for col in (col1, col2)]
    data.columns = data.columns.str.replace(r"(Bid|Ask).*?_", r"\1_", regex=True)

    # Calcular Mid, Spread y Rendimientos
    for asset in assets:
        data[f"Mid_{asset}"] = (data[f"Ask_{asset}"] + data[f"Bid_{asset}"]) / 2
        data[f"Spread_{asset}"] = (data[f"Ask_{asset}"] - data[f"Bid_{asset}"]) / data[f"Mid_{asset}"]
        data[f"Rend_{asset}"] = data[f"Mid_{asset}"].pct_change()

    # Calcular posiciones y pesos
    posiciones = np.array([data[f"Mid_{asset}"].iloc[-1] * tit for asset, tit in zip(assets, titulos)])
    total_posiciones = posiciones.sum()
    weights = posiciones / total_posiciones

    # Calcular rendimiento del portafolio
    rend_port = data[[f"Rend_{asset}" for asset in assets]].dot(weights).dropna()

    # Cálculo de VaR y CVaR
    var = np.percentile(rend_port, 100 - conf) if long else np.percentile(rend_port, conf)
    cvar = rend_port[rend_port < var].mean() if long else rend_port[rend_port > var].mean()
    
    # Crear DataFrame de VaR y CVaR
    var_df = pd.DataFrame({
        "Métrica": ["VaR", "CVaR"], 
        "Porcentual (%)": [var * 100, cvar * 100], 
        "Monetario ($)": [var * total_posiciones, cvar * total_posiciones]
    })

    # Cálculo de Costos de Liquidez (CL)
    CL = np.array([np.percentile(data[f"Spread_{asset}"].dropna(), 99) for asset in assets])
    CL_m = CL * posiciones

    # DataFrame de Costos de Liquidez
    CL_df = pd.DataFrame({
        "Activo": assets, 
        "Porcentual (%)": CL * 100, 
        "Monetario ($)": CL_m
    })

    # Cálculo de VaR y CVaR ajustados por liquidez
    var_apl = pd.DataFrame({
        "Métrica": ["VaR ApL", "CVaR ApL"], 
        "Porcentual (%)": [(var + CL.sum()) * 100, (cvar + CL.sum()) * 100], 
        "Monetario ($)": [var * total_posiciones + CL_m.sum(), cvar * total_posiciones + CL_m.sum()]
    })

    return display(var_df), display(CL_df), display(var_apl)

In [11]:
data = pd.read_excel('PruebaDatosVaRAjustado.xlsx', sheet_name=1).set_index('Date').sort_index()
assets = ['TB10Y', 'TB30Y']
titulos = [100000000, 90000000]
long = False
conf = 99

VaR_ApL(data, assets, titulos, long, conf)

,Métrica,Porcentual (%),Monetario ($)
0,VaR,1.6074,"351,914,316.5583"
1,CVaR,1.8068,"395,554,861.5311"


,Activo,Porcentual (%),Monetario ($)
0,TB10Y,0.0290,"3,240,128.4630"
1,TB30Y,0.0793,"8,511,584.1758"


,Métrica,Porcentual (%),Monetario ($)
0,VaR ApL,1.7158,"363,666,029.1972"
1,CVaR ApL,1.9151,"407,306,574.1699"


(None, None, None)